In [1]:
import sys 
import math
import gymnasium as gym
from gymnasium.wrappers.record_video import RecordVideo
from gymnasium.wrappers.record_episode_statistics import RecordEpisodeStatistics
import kidspuzzles
from stable_baselines3 import A2C

In [2]:
def linear_scheduler(initial_value):
    def func(progress_remaining):
        return initial_value * progress_remaining
    return func

def step_scheduler(initial_value, drop_interval, drop_factor):
    def func(progress_remaining):
        return initial_value * (drop_factor ** (int((1 - progress_remaining) / drop_interval)))
    return func

def cosine_annealing_scheduler(initial_value, T_max):
    def func(progress_remaining):
        return initial_value * (1 + math.cos(math.pi * progress_remaining / T_max)) / 2
    return func

In [3]:

n_digits = 10
# reward_clipped = -0.02
# reward_enter_target_area = 0.01
# reward_exit_target_area = -0.02

env = gym.make(
    'kidspuzzles/DigitsPuzzleEnv-v0', 
    render_mode = 'rgb_array', 
    n_digits = n_digits,
    # reward_clipped = reward_clipped,
    # reward_enter_target_area = reward_enter_target_area,
    # reward_exit_target_area = reward_exit_target_area
)

record_freq = 200

total_timesteps=5_000_000
initial_lr = 7e-4
lin_lr_scheduler = linear_scheduler(initial_lr)
step_lr_scheduler = step_scheduler(initial_value=initial_lr, drop_factor=0.5, drop_interval=0.33)
cos_lr_scheduler = cosine_annealing_scheduler(initial_lr, T_max=total_timesteps)

In [4]:

lr_scheduler = step_lr_scheduler
lr_name = "step_f0.5_i0.33"

In [5]:
    
video_folder = f"DigitsPuzzle-{n_digits}-{lr_name}"
env = RecordVideo(env, video_folder=f"videos/{video_folder}", name_prefix="training",
                episode_trigger=lambda x: x % record_freq == 0)

model = A2C(
    "MultiInputPolicy", 
    env, 
    ent_coef=0.01,
    n_steps=64,
    verbose=1, 
    device="mps", 
    learning_rate=lr_scheduler,
    tensorboard_log=f"./logs/a2c_digitspuzzle_nd{n_digits}_{lr_name}/"
)
model.learn(total_timesteps=total_timesteps)

Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs/a2c_digitspuzzle_nd10_step_f0.5_i0.2/A2C_2
MoviePy - Building video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-0.mp4.
MoviePy - Writing video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-0.mp4


-----------------------------------------
| rollout/              |               |
|    ep_len_mean        | 300           |
|    ep_rew_mean        | -176          |
| time/                 |               |
|    fps                | 234           |
|    iterations         | 100           |
|    time_elapsed       | 26            |
|    total_timesteps    | 6400          |
| train/                |               |
|    entropy_loss       | -3.64         |
|    explained_variance | -0.0021808147 |
|    learning_rate      | 0.0007        |
|    n_updates          | 99            |
|    policy_loss        | -20.6         |
|    value_loss         | 41.7          |
-----------------------------------------
-----------------------------------------
| rollout/              |               |
|    ep_len_mean        | 300           |
|    ep_rew_mean        | -170          |
| time/                 |               |
|    fps                | 255           |
|    iterations         | 200     

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-200.mp4


------------------------------------------
| rollout/              |                |
|    ep_len_mean        | 300            |
|    ep_rew_mean        | -150           |
| time/                 |                |
|    fps                | 272            |
|    iterations         | 1000           |
|    time_elapsed       | 234            |
|    total_timesteps    | 64000          |
| train/                |                |
|    entropy_loss       | -3.63          |
|    explained_variance | -1.9788742e-05 |
|    learning_rate      | 0.0007         |
|    n_updates          | 999            |
|    policy_loss        | -9.67          |
|    value_loss         | 17.1           |
------------------------------------------
-----------------------------------------
| rollout/              |               |
|    ep_len_mean        | 300           |
|    ep_rew_mean        | -147          |
| time/                 |               |
|    fps                | 273           |
|    iterations  

...

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-39600.mp4


--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 46.1       |
|    ep_rew_mean        | 4.27       |
| time/                 |            |
|    fps                | 273        |
|    iterations         | 46600      |
|    time_elapsed       | 10888      |
|    total_timesteps    | 2982400    |
| train/                |            |
|    entropy_loss       | -2.35      |
|    explained_variance | 0.97471917 |
|    learning_rate      | 8.3e-06    |
|    n_updates          | 46599      |
|    policy_loss        | 0.358      |
|    value_loss         | 0.581      |
--------------------------------------
MoviePy - Building video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-39800.mp4.
MoviePy - Writing video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-39800.mp4



MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-39800.mp4


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 46.3      |
|    ep_rew_mean        | 4.8       |
| time/                 |           |
|    fps                | 273       |
|    iterations         | 46700     |
|    time_elapsed       | 10911     |
|    total_timesteps    | 2988800   |
| train/                |           |
|    entropy_loss       | -2.24     |
|    explained_variance | 0.9239767 |
|    learning_rate      | 8.3e-06   |
|    n_updates          | 46699     |
|    policy_loss        | -1.35     |
|    value_loss         | 0.973     |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 46.8      |
|    ep_rew_mean        | 4.25      |
| time/                 |           |
|    fps                | 273       |
|    iterations         | 46800     |
|    time_elapsed       | 10934     |
|    total_timesteps    | 2995200   |
| train/    

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10-step_f0.5_i0.2/training-episode-40000.mp4


In [6]:
import time
vec_env = model.get_env()
observations = vec_env.reset()
reward_sum = 0
for _ in range(50):
    action, _state = model.predict(observations)
    observations, reward, terminated, info = vec_env.step(action)
    vec_env.render("human")
    time.sleep(5)

    reward_sum += reward
    if terminated:
        observations = vec_env.reset()

print("Total reward: ", reward_sum)

2025-02-09 12:48:27.298 python[40735:2934412] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-09 12:48:27.298 python[40735:2934412] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Total reward:  [2.5222225]


In [7]:
env.close()